In [131]:
import numpy as np
import pandas as pd
import pickle

In [132]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [133]:
df = df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1)

In [134]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [135]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
label_encod_gender = LabelEncoder()
one_hot_encod_geo = OneHotEncoder()
scaler_reg = StandardScaler()

In [136]:
df['Gender'] = label_encod_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [137]:
label_encod_gender.classes_

array(['Female', 'Male'], dtype=object)

In [138]:
encoded_geo = one_hot_encod_geo.fit_transform(df[['Geography']]).toarray()

In [139]:
encoded_geo

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [140]:
one_hot_encod_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [141]:
one_hot_encod_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [142]:
encoded_geo_df = pd.DataFrame(encoded_geo,columns=one_hot_encod_geo.get_feature_names_out(['Geography']))
encoded_geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [143]:
df = pd.concat([df.drop(columns=['Geography'],axis=1),encoded_geo_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [144]:
X = df.drop(columns=['EstimatedSalary'])
y = df['EstimatedSalary']

In [145]:
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0


In [146]:
y.head()

0    101348.88
1    112542.58
2    113931.57
3     93826.63
4     79084.10
Name: EstimatedSalary, dtype: float64

In [147]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [148]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,0,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,0,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,1,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,1,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,0,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,0,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,1,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,1,1.0,0.0,0.0


In [149]:
X_test

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
6252,596,1,32,3,96709.07,2,0,0,0,0.0,1.0,0.0
4684,623,1,43,1,0.00,2,1,1,0,1.0,0.0,0.0
1731,601,0,44,4,0.00,2,1,0,0,0.0,0.0,1.0
4742,506,1,59,8,119152.10,2,1,1,0,0.0,1.0,0.0
4521,560,0,27,7,124995.98,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6412,602,0,53,5,98268.84,1,0,1,1,0.0,1.0,0.0
8285,609,1,25,10,0.00,1,0,1,0,1.0,0.0,0.0
7853,730,0,47,7,0.00,1,1,0,1,1.0,0.0,0.0
1095,692,1,29,4,0.00,1,1,0,1,1.0,0.0,0.0


In [150]:
X_train = scaler_reg.fit_transform(X_train)
X_test = scaler_reg.transform(X_test)

In [151]:
with open("reg_label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encod_gender,file)

with open("reg_one_hot_encoder_geo.pkl",'wb') as file:
    pickle.dump(one_hot_encod_geo,file)

with open("reg_scaler.pkl",'wb') as file:
    pickle.dump(scaler_reg,file)

## ANN Regression

In [152]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [153]:
(X_train.shape[1],)

(12,)

In [154]:
model_reg = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)            # default activation = linear

])

d:\AI\Projects\DL\ANNproject\myvenv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [155]:
model_reg.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [156]:
model_reg.compile(loss='mean_absolute_error',optimizer='adam',metrics=['mae'])

In [157]:
early_stopping_callbacks = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [158]:
import datetime

logdir = "reglog/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=logdir,histogram_freq=1)

In [159]:
model_reg.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs = 70,
    callbacks = [early_stopping_callbacks,tensorflow_callbacks]
)

Epoch 1/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100390.1562 - mae: 100390.1562 - val_loss: 98562.9375 - val_mae: 98562.9375
Epoch 2/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 99793.0547 - mae: 99793.0547 - val_loss: 97351.8594 - val_mae: 97351.8594
Epoch 3/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 97654.1250 - mae: 97654.1250 - val_loss: 94152.2109 - val_mae: 94152.2109
Epoch 4/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 93238.6797 - mae: 93238.6797 - val_loss: 88547.9219 - val_mae: 88547.9219
Epoch 5/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 86560.6562 - mae: 86560.6562 - val_loss: 81075.2031 - val_mae: 81075.2031
Epoch 6/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 78318.4609 - mae: 78318.4609 - val_loss: 72748.3125 - val_mae: 72748.3125
Epoch 7/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69789.8984 - mae: 69789.8984 - val_loss: 64931.5234 - val_mae: 64931.5234
Epoch 8/70
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 

In [160]:
model_reg.save("model_reg.h5")

In [161]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [162]:
%tensorboard --logdir reglog/fit20250828-112508

Reusing TensorBoard on port 6006 (pid 19416), started 0:21:42 ago. (Use '!kill 19416' to kill it.)